In [1]:
!conda install --offline /kaggle/input/track4-train/*.tar.bz2


######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
######################################################################## | 100% 
###########################

In [2]:
# !ls /kaggle/input/track4-train/models

In [3]:
# [0.6382310415028003, 0.6189348573669843, 0.594305478150372, 0.6456368418815037, 0.650946396469057, 0.6610530600887038]
# 0.6348512792432368
# Full validation metric: 0.6373838583629989

# Counter({1: 47225, 0: 42655})
# ROC AUC metric: 0.6831401875473259
# Accuracy: 0.6177681352914998
# Full target metric: 0.5901076765112288
# Full target metric fixed: 0.5898723615115555
# Target metric by center_id:
# Center_id 1: 0.5837030889916592
# Center_id 2: 0.5128716159021234
# Center_id 3: 0.6234127532907505
# Center_id 4: 0.5803120005321822
# Center_id 5: 0.633916901390537
# Center_id 6: 0.6509644227059665
# Center_id 7: 0.5673793620392944
# Center_id 8: 0.6490039016757818
# Center_id 9: 0.529810187493138
# Center_id 10: 0.5777616326658479
# Center_id 11: 0.5891695748590473

In [4]:
# import pandas as pd
# from collections import Counter


# data = pd.read_csv('/kaggle/input/mayo-clinic-strip-ai/train.csv')
# print(Counter(data['center_id'].tolist()))

In [5]:
BAD_IMAGE_IDS = ['5adc4c_0', '7b9aaa_0', 'bb06a5_0', 'e26a04_0', '280c26_0'] + \
                ['4ae44b_0', '53e66f_0', '7c2c2f_0', '74a450_1']

BLOCK_SIZE = 28
BLOCKS_PER_CROP = 8
CROP_SIZE = BLOCK_SIZE * BLOCKS_PER_CROP
BLOCK_THR = 90
CROP_THR = 0.6
MAX_CROPS_PER_IMAGE = 20
IMAGES_PER_SAMPLE = 4
EPOCHS_NUM = 10
SCALE_FACTOR = 24
TEST_SAMPLE_DUPL_RATE = 20
TRAIN_SAMPLE_DUPL_RATE = 4

In [6]:
import random
from collections import defaultdict
from typing import List

import numpy as np
import torch
from PIL import Image


class ClotImageDataset(torch.utils.data.Dataset):
    def __init__(
            self,
            image_ids: List[str],
            labels: List[str],
            image_crops: List[List[np.ndarray]],
            seed: int,
            is_test: bool,
            transformations,
    ):
        self.image_ids = image_ids
        self.labels = [float(label == 'CE') for label in labels]
        self.image_crops = image_crops
        self.seed = seed
        self.is_test = is_test
        self.transformations = transformations

        if not self.is_test:
            np.random.seed(self.seed)

            label_to_indices = defaultdict(list)
            for i, (label, crops) in enumerate(zip(self.labels, self.image_crops)):
                if len(crops) > 0:
                    label_to_indices[label].append(i)

            max_size = TRAIN_SAMPLE_DUPL_RATE * max(len(indices) for indices in label_to_indices.values())

            self.sample_ids = []
            for i, indices in enumerate(label_to_indices.values()):
                np.random.shuffle(indices)
                while len(self.sample_ids) < max_size * (i + 1):
                    req_size = min(len(indices), max_size * (i + 1) - len(self.sample_ids))
                    self.sample_ids += indices[:req_size]
        else:
            self.sample_ids = []
            for _ in range(TEST_SAMPLE_DUPL_RATE):
                self.sample_ids.extend(list(range(len(self.image_ids))))

        self.image_index_ids = []
        sample_id_to_image_index = defaultdict(int)
        for sample_id in self.sample_ids:
            self.image_index_ids.append(sample_id_to_image_index[sample_id])
            image_crops_cnt = len(self.image_crops[sample_id])
            if image_crops_cnt:
                sample_id_to_image_index[sample_id] = (sample_id_to_image_index[sample_id] + 1) % image_crops_cnt

    def __len__(self):
        return len(self.sample_ids)

    def __getitem__(self, idx):
        if self.is_test:
            np.random.seed(self.seed + idx)
            random.seed(self.seed + idx)
            torch.manual_seed(self.seed + idx)
        idx, image_index = self.sample_ids[idx], self.image_index_ids[idx]
        if len(self.image_crops[idx]) == 0:
            return (
                self.transformations(Image.fromarray(np.zeros((224, 224, 3)).astype(np.uint8))),
                torch.tensor(self.labels[idx]),
                self.image_ids[idx],
            )
        # image_index = np.random.randint(0, len(self.image_crops[idx]))
        return (
            self.transformations(self.image_crops[idx][image_index]),
            torch.tensor(self.labels[idx]),
            self.image_ids[idx],
        )


def get_loader(
        image_ids: List[str],
        labels: List[str],
        image_crops: List[List[np.ndarray]],
        seed: int,
        is_test: bool,
        transformations,
        shuffle: bool,
        batch_size: int,
        num_workers: int
):
    dataset = ClotImageDataset(
        image_ids, labels, image_crops, seed, is_test, transformations,
    )
    return torch.utils.data.DataLoader(dataset, shuffle=shuffle, batch_size=batch_size, num_workers=num_workers)

In [7]:
import gc
import os
from time import time
from typing import List, Tuple

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import pyvips
import cv2
import matplotlib.pyplot as plt


class DataPreparation:
    def __init__(self, visualize: bool = False, seed: int = 42):
        self.visualize = visualize
        self.seed = seed

        train_metadata = pd.read_csv('/kaggle/input/mayo-clinic-strip-ai/train.csv')
        train_metadata = list(zip(
            train_metadata['image_id'].tolist(),
            train_metadata['label'].tolist(),
            train_metadata['center_id'].tolist(),
        ))
        self.train = self._filter_bad_images(train_metadata)
        self.all_center_ids = sorted(list({center_id for _, _, center_id in self.train}))

        other_metadata = pd.read_csv('/kaggle/input/mayo-clinic-strip-ai/other.csv').query('label == \'Other\'')
        other_metadata = list(zip(
            other_metadata['image_id'].tolist(),
            ['LAA' for _ in range(other_metadata.shape[0])],
            [-1 for _ in range(other_metadata.shape[0])],
        ))
        self.other = self._filter_bad_images(other_metadata)
        
#         self.test = self.train
        test_metadata = pd.read_csv('/kaggle/input/mayo-clinic-strip-ai/test.csv')
        self.test = list(zip(
            test_metadata['image_id'].tolist(),
            ['Unknown' for _ in range(test_metadata.shape[0])],
            test_metadata['center_id'].tolist(),
        ))

    @staticmethod
    def _filter_bad_images(data: List[Tuple]) -> List[Tuple]:
        return [
            (image_id, label, center_id)
            for image_id, label, center_id in data
            if image_id not in BAD_IMAGE_IDS
        ]

    @staticmethod
    def _add_rect_to_numpy(image: np.ndarray, x: int, y: int, size: int, thickness: int) -> None:
        image[x:x + size, y:y + thickness] = (0, 0, 0)
        image[x:x + thickness, y:y + size] = (0, 0, 0)
        image[x:x + size, y + size:y + size + thickness] = (0, 0, 0)
        image[x + size:x + size + thickness, y:y + size] = (0, 0, 0)

    @staticmethod
    def _get_blocks_map(image: np.ndarray) -> np.ndarray:
        pixels_diff = np.sum((image[:-1, :, :] - image[1:, :, :]) ** 2, axis=2)
        pixels_diff = np.cumsum(np.cumsum(pixels_diff, axis=0), axis=1)
        blocks_map = np.zeros((
            (image.shape[0] + BLOCK_SIZE - 1) // BLOCK_SIZE,
            (image.shape[1] + BLOCK_SIZE - 1) // BLOCK_SIZE,
        ))
        for x in range(0, pixels_diff.shape[0], BLOCK_SIZE):
            for y in range(0, pixels_diff.shape[1], BLOCK_SIZE):
                nx = min(x + BLOCK_SIZE, pixels_diff.shape[0])
                ny = min(y + BLOCK_SIZE, pixels_diff.shape[1])
                block_sum = int(pixels_diff[nx - 1, ny - 1])
                if x:
                    block_sum -= int(pixels_diff[x - 1, ny - 1])
                if y:
                    block_sum -= int(pixels_diff[nx - 1, y - 1])
                if x and y:
                    block_sum += int(pixels_diff[x - 1, y - 1])
                blocks_map[x // BLOCK_SIZE][y // BLOCK_SIZE] = \
                    (block_sum / BLOCK_SIZE / BLOCK_SIZE) > BLOCK_THR
        return blocks_map

    def _generate_crops_positions(
            self,
            image: np.ndarray,
            crop_thr: float,
    ) -> Tuple[List[Tuple[int, int]], np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        blocks_map = self._get_blocks_map(image)

        if self.visualize:
            for i in range(blocks_map.shape[0]):
                for j in range(blocks_map.shape[1]):
                    if blocks_map[i][j]:
                        self._add_rect_to_numpy(
                            image,
                            i * BLOCK_SIZE,
                            j * BLOCK_SIZE,
                            BLOCK_SIZE,
                            1,
                        )

        good_crops_starts = []
        for x in range(0, image.shape[0] - CROP_SIZE + 1, BLOCK_SIZE):
            for y in range(0, image.shape[1] - CROP_SIZE + 1, BLOCK_SIZE):
                _x, _y = x // BLOCK_SIZE, y // BLOCK_SIZE
                crop_sum = blocks_map[_x:_x + BLOCKS_PER_CROP, _y:_y + BLOCKS_PER_CROP].sum()
                if crop_sum > BLOCKS_PER_CROP * BLOCKS_PER_CROP * crop_thr:
                    good_crops_starts.append((x, y))

        if self.visualize:
            for x, y in good_crops_starts:
                self._add_rect_to_numpy(image, x, y, CROP_SIZE, 1)

        return good_crops_starts

    @staticmethod
    def _process_crop(crop: np.ndarray) -> np.ndarray:
        return crop

    def _create_crops(
        self,
        image: np.ndarray,
        crops_starts: List[Tuple[int]],
    ) -> List[np.ndarray]:
        return [
            Image.fromarray(
                self._process_crop(
                    image[x:x + CROP_SIZE, y:y + CROP_SIZE],
                )
            )
            for x, y in crops_starts
        ]

    @staticmethod
    def _get_unique_crops(crop_starts: List[Tuple[int, int]], order) -> List[Tuple[int, int]]:
        def inter_size_1d(a: int, b: int, c: int, d: int) -> int:
            return max(0, min(b, d) - max(a, c))

        def inter_size_2d(crop_start_1: Tuple[int, int], crop_start_2: Tuple[int, int]) -> int:
            return inter_size_1d(
                crop_start_1[0], crop_start_1[0] + CROP_SIZE,
                crop_start_2[0], crop_start_2[0] + CROP_SIZE,
            ) * inter_size_1d(
                crop_start_1[1], crop_start_1[1] + CROP_SIZE,
                crop_start_2[1], crop_start_2[1] + CROP_SIZE,
            )

        crop_starts_sorted = sorted(crop_starts, key=order)
        final_crop_starts = []
        for crop_start in crop_starts_sorted:
            if any(
                    inter_size_2d(crop_start, crop_start_prev) > CROP_SIZE * CROP_SIZE // 2
                    for crop_start_prev in final_crop_starts
            ):
                continue
            final_crop_starts.append(crop_start)
        return final_crop_starts
    
    @staticmethod
    def _read_and_resize_image(image_id: str, base_image_path: str) -> np.ndarray:       
        image_path = os.path.join(base_image_path, f'{image_id}.tif')
        image = pyvips.Image.new_from_file(image_path, access='sequential')
        return image.resize(1.0 / SCALE_FACTOR).numpy()
    
    def prepare_crops(
            self,
            image_ids: List[int],
            base_image_path: str,
    ) -> Tuple[List[List[np.ndarray]], List[List[Tuple[int]]], List[Tuple[np.ndarray, np.ndarray]]]:
        np.random.seed(self.seed)
        image_crops = []
        image_crops_indices = []
        for image_id in tqdm(image_ids):
            start_time = time()
            image = self._read_and_resize_image(image_id, base_image_path)
            gc.collect()
            print(f'Rescaling done in {time() - start_time} seconds. Image shape is {image.shape}')
            found_flag = False
            for crop_thr in np.arange(CROP_THR, -0.1, -0.1):
                good_crops_starts = self._generate_crops_positions(image, crop_thr)
                if len(good_crops_starts) < IMAGES_PER_SAMPLE:
                    print('Bad image', image_id, 'crop_thr', crop_thr, 'only', len(good_crops_starts))
                    continue

                good_crops_starts_unique = []
                for order in [
                    lambda x: (x[0], x[1]),
                    lambda x: (-x[0], -x[1]),
                ]:
                    good_crops_starts_unique.extend(self._get_unique_crops(good_crops_starts, order))
                good_crops_starts_unique = list(set(good_crops_starts_unique))

                if len(good_crops_starts_unique) < IMAGES_PER_SAMPLE:
                    print('Bad image', image_id, 'crop_thr', crop_thr, 'only', len(good_crops_starts_unique))
                    continue

                good_crops_starts_sample_ids = np.random.choice(
                    list(range(len(good_crops_starts_unique))),
                    min(len(good_crops_starts_unique), MAX_CROPS_PER_IMAGE),
                    replace=False,
                )
                good_crops_starts_sample = np.array(good_crops_starts_unique)[good_crops_starts_sample_ids]
                image_crops_indices.append(good_crops_starts_sample)
                image_crops.append(self._create_crops(image, good_crops_starts_sample))
                found_flag = True
                break
            if not found_flag:
                image_crops_indices.append([])
                image_crops.append([])
                print('No crops was found')
            print(f'Done {image_id} in {time() - start_time} seconds')
        gc.collect()
        return image_crops, image_crops_indices

    def process_train(
            self
    ) -> Tuple[List[List[np.ndarray]], List[List[Tuple[int]]]]:
        return self.prepare_crops(
            [image_id for image_id, _, _ in self.train],
            '/kaggle/input/mayo-clinic-strip-ai/train/',
        )

    def process_other(
            self
    ) -> Tuple[List[List[np.ndarray]], List[List[Tuple[int]]]]:
        return self.prepare_crops(
            [image_id for image_id, _, _ in self.other],
            '/kaggle/input/mayo-clinic-strip-ai/other/',
        )

    def process_test(
        self
    ) -> Tuple[List[List[np.ndarray]], List[List[Tuple[int]]]]:
        return self.prepare_crops(
            [image_id for image_id, _, _ in self.test],
            '/kaggle/input/mayo-clinic-strip-ai/test',
        )

In [8]:
from torchvision import transforms


train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    # transforms.RandomResizedCrop((224, 224), scale=(0.5, 1.0), ratio=(1.0, 1.0)),
    transforms.RandomAdjustSharpness(sharpness_factor=2, p=1.0),
    transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
    transforms.ColorJitter(brightness=0.2, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transforms = transforms.Compose([
    # transforms.RandomResizedCrop((224, 224), scale=(0.5, 1.0), ratio=(1.0, 1.0)),
    transforms.RandomAdjustSharpness(sharpness_factor=2, p=1.0),
    transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
    transforms.ColorJitter(brightness=0.2, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [9]:
from collections import defaultdict

import numpy as np


def _group_by_patients(y_true, y_pred, image_ids):
    patients = [image_id.split('_')[0] for image_id in image_ids]
    patient_to_y_true, patient_to_y_pred = defaultdict(list), defaultdict(list)
    for y, y_hat, patient in zip(y_true, y_pred, patients):
        patient_to_y_true[patient].append(y)
        patient_to_y_pred[patient].append(y_hat)
    patient_to_y_true = {
        patient: np.mean(y_true)
        for patient, y_true in patient_to_y_true.items()
    }
    patient_to_y_pred = {
        patient: np.mean(y_pred).tolist()
        for patient, y_pred in patient_to_y_pred.items()
    }
    y_true, y_pred, patients = [], [], []
    for patient, y in patient_to_y_true.items():
        y_true.append(y)
        y_pred.append(patient_to_y_pred[patient])
        patients.append(patient)
        
    return y_true, np.array([[1 - p, p] for p in y_pred]), patients


def get_target_metric(y_true, y_pred, image_ids):
    return _weighted_mc_log_loss(*_group_by_patients(y_true, y_pred, image_ids)[:2])


def _weighted_mc_log_loss(y_true, y_pred, epsilon=1e-15):
    class_cnt = [sum(int(val == cl) for val in y_true) for cl in range(2)]
    w = [0.5 for _ in range(2)]
    return -sum(
        w[cl] * sum(
            (y == cl) / class_cnt[cl] * np.log(max(min(y_hat, 1 - epsilon), epsilon))
            for y, y_hat in zip(y_true, y_pred[:, cl])
        )
        for cl in range(2)
    ) / sum(w[cl] for cl in range(2))


In [10]:
import torch
import torch.nn as nn
from torchvision import models


class ClotModelSingle(nn.Module):
    def __init__(self, encoder_model):
        super().__init__()

        if encoder_model == 'effnet_b0':
            base_model = models.efficientnet_b0(pretrained=True)
            self.model = base_model.features
            in_features_cnt = base_model.classifier[1].in_features
        elif encoder_model == 'resnet18':
            base_model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
            self.model = nn.Sequential(*list(base_model.children())[:-2])
            in_features_cnt = list(base_model.children())[-1].in_features
        elif encoder_model == 'regnet_x_1_6gf':
            base_model = models.regnet_x_1_6gf(weights=models.RegNet_X_1_6GF_Weights.IMAGENET1K_V2)
            self.model = nn.Sequential(base_model.stem, base_model.trunk_output)
            in_features_cnt = base_model.fc.in_features
        else:
            raise Exception('Incorrect encoder name')

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(output_size=1),
            nn.Flatten(),
            nn.Linear(in_features_cnt, 1),
            nn.Sigmoid(),
        )

    def freeze_encoder(self, flag):
        for param in self.model.parameters():
            param.requires_grad = not flag

    def forward(self, x):
        return self.head(self.model(x))

    def save(self, model_path):
        weights = self.state_dict()
        torch.save(weights, model_path)

    def load(self, model_path):
        weights = torch.load(model_path, map_location='cpu')
        self.load_state_dict(weights)


In [11]:
def update_final_prediction_with_train_data(submission: pd.DataFrame) -> pd.DataFrame:
    train_data = pd.read_csv('/kaggle/input/mayo-clinic-strip-ai/train.csv')
    patient_id_to_label = {
        image_id.split('_')[0]: label
        for image_id, label in zip(train_data['image_id'].tolist(), train_data['label'].tolist())
    }
    submission['CE'] = [
        pred if patient_id not in patient_id_to_label else float(patient_id_to_label[patient_id] == 'CE')
        for patient_id, pred in zip(submission['patient_id'].tolist(), submission['CE'].tolist())
    ]
    submission['LAA'] = [
        pred if patient_id not in patient_id_to_label else float(patient_id_to_label[patient_id] == 'LAA')
        for patient_id, pred in zip(submission['patient_id'].tolist(), submission['LAA'].tolist())
    ]
    return submission

In [12]:
from __future__ import print_function, division

import os
import pickle
from collections import Counter
from typing import List

import ssl
import torch
import torch.backends.cudnn as cudnn
from sklearn.metrics import roc_auc_score
from tqdm import tqdm


ssl._create_default_https_context = ssl._create_unverified_context
cudnn.benchmark = True

TEST_BATCH_SIZE = 16
DUMPED_DATALOADER_PATH = '/kaggle/input/track-4-dataprep/data_loaders.pkl'


def _get_model_path_by_center_id(folder_name: str, center_id: str) -> str:
    for file_name in os.listdir(folder_name):
        if not file_name.endswith('.h5'):
            continue
        if file_name.split('_')[2] == center_id:
            return os.path.join(folder_name, file_name)
    raise Exception(f'Model for center id {center_id} was not found in folder {folder_name}')


def _explode_image_ids(image_ids: List[str], factor: int) -> List[str]:
    exploded_image_ids = []
    for start_id in range(0, len(image_ids), TEST_BATCH_SIZE):
        end_id = min(start_id + TEST_BATCH_SIZE, len(image_ids))
        for _ in range(factor):
            exploded_image_ids.extend(image_ids[start_id:end_id])
    return exploded_image_ids


data_prep = DataPreparation()

image_crops, _ = data_prep.process_test()
# with open(DUMPED_DATALOADER_PATH, 'rb') as file:
#     image_crops, _ = pickle.load(file)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dataloader = get_loader(
    [image_id for image_id, _, _ in data_prep.test],
    [label for _, label, _ in data_prep.test],
    image_crops,
    seed=41,
    is_test=True,
    transformations=test_transforms,
    shuffle=False,
    batch_size=TEST_BATCH_SIZE,
    num_workers=2,
)

models = [
    torch.load(_get_model_path_by_center_id(
        '../input/track4-trains',
        center_id,
    ), map_location=torch.device('cpu')).to(device)
    for center_id in ['1.5', '10.3', '11', '4', '6.2.8.9', '7']
]
for model in models:
    model.eval()

with torch.no_grad():
    y_hat, y, image_ids = [], [], []
    for image, label, image_id in tqdm(dataloader):
        image = image.to(device)
        label = label.cpu().detach().numpy().tolist()
        for model in models:
            y_hat.extend(model.forward(image).squeeze().cpu().detach().numpy().tolist())
            y.extend(label)
            image_ids.extend(image_id)
            
bad_image_ids = {
    image_id
    for image_id, crops in zip([image_id for image_id, _, _ in data_prep.test], image_crops)
    if len(crops) == 0 
}
y_hat_fixed = [
    0.5 if image_id in bad_image_ids else p
    for p, image_id in zip(y_hat, image_ids)
]

labels, preds, patients = _group_by_patients(y, y_hat_fixed, image_ids)
result = pd.DataFrame({
    'patient_id': patients,
    'CE': [pair[1].round(6) for pair in preds],
    'LAA': [pair[0].round(6) for pair in preds],
})
print(result)
result = update_final_prediction_with_train_data(result)
print(result)
result.to_csv('submission.csv', index=False)
# print(Counter([int(p > 0.5) for p in y_hat_fixed]))
# print('ROC AUC metric:', roc_auc_score(y, y_hat_fixed))
# accuracy = sum([int(int(p > 0.5) == label) for p, label in zip(y_hat_fixed, y)]) / len(y)
# print('Accuracy:', accuracy)
# image_id_to_center_id = {image_id: center_id for image_id, _, center_id in data_prep.test}
# target_metric = get_target_metric(
#     y,
#     y_hat,
#     image_ids,
# )
# print('Full target metric:', target_metric)
# target_metric = get_target_metric(
#     y,
#     y_hat_fixed,
#     image_ids,
# )
# print('Full target metric fixed:', target_metric)
# print('Target metric by center_id:')
# for center_id in range(1, 12):
#     sub_y = [label for label, image_id in zip(y, image_ids) if image_id_to_center_id[image_id] == center_id]
#     sub_y_hat = [pred for pred, image_id in zip(y_hat_fixed, image_ids) if image_id_to_center_id[image_id] == center_id]
#     sub_image_ids = [image_id for image_id in image_ids if image_id_to_center_id[image_id] == center_id]
#     print(f'Center_id {center_id}: {get_target_metric(sub_y, sub_y_hat, sub_image_ids)}')

  0%|          | 0/4 [00:00<?, ?it/s]

Rescaling done in 44.03065800666809 seconds. Image shape is (2533, 1417, 3)


 25%|██▌       | 1/4 [00:44<02:12, 44.31s/it]

Done 006388_0 in 44.31237602233887 seconds


 50%|█████     | 2/4 [00:47<00:39, 19.87s/it]

...

Rescaling done in 12.525835037231445 seconds. Image shape is (2575, 636, 3)
Done 00c058_0 in 12.608379125595093 seconds
Rescaling done in 44.161428451538086 seconds. Image shape is (1106, 2326, 3)
Bad image 01adc5_0 crop_thr 0.6 only 1


100%|██████████| 4/4 [01:44<00:00, 26.02s/it]

Done 01adc5_0 in 44.39000487327576 seconds



100%|██████████| 5/5 [00:06<00:00,  1.38s/it]

  patient_id        CE       LAA
0     006388  0.469765  0.530235
1     008e5c  0.519275  0.480725
2     00c058  0.508980  0.491020
3     01adc5  0.540291  0.459709
  patient_id   CE  LAA
0     006388  1.0  0.0
1     008e5c  1.0  0.0
2     00c058  0.0  1.0
3     01adc5  0.0  1.0
